 ## Covid-19 Dashboard

The following dashboard aims to successfully implement the following functionalities:


1. Acquire pandemic data from the Public Health England Application Programming Interface (API) through the Software Development Kit
2.  Visualise and wrangle data in the form of one graph
3.  Enable a user to trigger the API access by clicking on a button, and providing at least a functional interactive control (in the form of a widget) for the graph


In [20]:
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [21]:
%matplotlib inline
plt.rcParams['figure.dpi'] = 100

In [22]:
data = {}
with open("agedistribution.json", "rt") as INFILE:
    data=json.load(INFILE)

## Aquiring Data and Visualisation

The dashboard represents the number of COVID-19 cases for males and females based on their age range. 
The first step was to access data through the Public Health England through a web-based Application Programming Interface (API). This often involves the excange of information in JSON format. 

Public Health Englanda offers a Python Software Devekopment Kit (SDK). This is a Python wrapper that facilitates access to the API by building requests.

Next, data was wrangled in order to get to a DataFrame with the age bins as an index, and values for males and females as its columns.

In [23]:
def wrangle_data(rawdata):
    
    datadic=rawdata['data'][0]

    males=datadic['males']
    females=datadic['females']

    ageranges=[x['age'] for x in males] 

    def min_age(agerange):
        agerange=agerange.replace('+','') 
        start=agerange.split('_')[0]
        return int(start)

    ageranges.sort(key=min_age)
    age_df=pd.DataFrame(index=ageranges, columns=['males','females', 'total'])

    for entry in males: 
        ageband=entry['age'] 
        age_df.loc[ageband, 'males']=entry['value']

    for entry in females:
        ageband=entry['age']
        age_df.loc[ageband, 'females']=entry['value']

    age_df['total']=age_df['males']+age_df['females']

    return age_df

age_df=wrangle_data(data)

In [24]:
  
def access_api():
    filters = [
    'areaType=nation',
    'areaName=England'
    ]

    structure = {
    "males": "maleCases",
    "females": "femaleCases"
    }
    api = Cov19API(filters=filters, structure=structure)
    agedistribution=api.get_json()
    return agedistribution




def api_button_callback(button):
    apidata=access_api()
    global age_df
    age_df=wrangle_data(apidata)
    refresh_graph()
    apibutton.icon="check"

    
apibutton=wdg.Button(
    description='Refresh data', 
    disabled=False,
    button_style='success', 
    tooltip="'Click to download current Public Health England data'",
    icon='refresh'
)
    

## Age Distribution Graph


Finally, interactive controls were added to the graph using the ipywidgets library, a Graphical User Interface (GUI) library that runs in a notebook.

This interactive graph represents the number of COVID-19 cases in England for males and females by dividing the data based on their age. By adding an interactive control, allows to select any combination of the males, females or total columns.

The refresh button forces a redraw of the graph which is useful when the data have been updated.

In [25]:
agecols=wdg.SelectMultiple(
    options=['males', 'females', 'total'], 
    value=['males'], 
    rows=3, 
    description='Sex',
    disabled=False
)

def refresh_graph():
    current=agecols.value[0]
    if current==agecols.options[0]:
        other=agecols.options[1]
    else:
        other=agecols.options[0]
    agecols.value=(other,) 
    agecols.value=(current,) 


def age_graph(graphcolumns):
    ncols=len(graphcolumns)
    if ncols>0:
        age_df.plot(kind='bar', y=list(graphcolumns)) 
        plt.title("Distribution of COVID-19 cases for males and females based on age range")
        plt.xlabel("Age Range")
        plt.ylabel("Number of COVID-19 cases")
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")
    
    
output=wdg.interactive_output(age_graph, {'graphcolumns': agecols})

apibutton.on_click(api_button_callback) 
display(apibutton)

display(agecols, output)

Button(button_style='success', description='Refresh data', icon='refresh', style=ButtonStyle(), tooltip="'Clic…

SelectMultiple(description='Sex', index=(0,), options=('males', 'females', 'total'), rows=3, value=('males',))

Output()

**Author and Copyright Notice** (C) Patricia Dominik, 27/11/2020 (bt17329@qmul.ac.uk). All rights reserved. *Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*